In [ ]:
import sys
import os

# Get the path of the project root directory 
# Assumes you are running jupyter lab from inside the 'earth-intelligence-platform' directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to the system path so Python can find 'eintelligence'
if project_root not in sys.path:
    sys.path.append(project_root)
    
print(f"Project root added to path: {project_root}")

In [ ]:
import matplotlib.pyplot as plt
from datetime import date
from pathlib import Path

from eintelligence.data_prep.fetch import get_sentinel2_data, load_sentinel2_bands, calculate_ndvi
from eintelligence.data_prep.save import save_multiband_to_geotiff
from eintelligence.data_prep.tiling import tile_to_cog

In [ ]:
TEST_LAT, TEST_LON = 36.5, -120.5 # Central California
START_DATE = date(2023, 7, 1)
END_DATE = date(2023, 8, 1)
BANDS = ("B02","B03","B04","B08") 

# find the best scene
item = get_sentinel2_data(TEST_LAT, TEST_LON, START_DATE, END_DATE)

In [ ]:
# tiling 

scene_id = item.id.replace(":", "_")
tiles_dir = Path(project_root) / "data" / scene_id / "tiles_s2"
tiles_dir.mkdir(parents=True, exist_ok=True)

from eintelligence.data_prep.tiler_streaming import tile_stac_item_to_cogs
manifest_path = tile_stac_item_to_cogs(
    item,
    bands=BANDS,
    out_dir=tiles_dir,
    tile_size=512,
    stride=256,          # overlap if you’ll train CNNs
    min_valid_fraction=0.3,
    web_optimized=True,  # False if you want lossless DEFLATE tiles
)
print("Manifest:", manifest_path)